# Medication Analysis via Neo4j



## Load data to Neo4j

In [14]:
from neo4j import GraphDatabase, Query
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget

# Connect to your Neo4j instance
uri = "neo4j+s://2121873f.databases.neo4j.io"
driver = GraphDatabase.driver(uri, auth=("neo4j", "FRP5W8toH8feYqDBZPOUo8eojUsW4k8pfAKVH6yTVdQ"))

# Cypher query to load data to Neo4j
delete_data = Query("""
//Delete All
MATCH (n)
DETACH DELETE n;
""")

load_nodes = Query("""
//Load All
LOAD CSV WITH HEADERS FROM "https://docs.google.com/spreadsheets/d/1hCPlV7HiiYhy6MMnbzkPv0DRmFjJDe4lVdzR9NBnIKo/export?format=csv" AS row
MERGE (n:Node {id: row.Name, class: row.Class, uri_class: row.URI_class, uri_root: row.URI_root, epa_label: row.epa_label});""")

load_relationships = Query("""
//Load Rels with Label
LOAD CSV WITH HEADERS FROM "https://docs.google.com/spreadsheets/d/1PaYa9_Nlf-6FUPaOWpKJpg7gdlkTAQb2StwaeB5CSi0/export?format=csv" AS row
MATCH (source:Node {id: row.Source_Node})
MATCH (target:Node {id: row.Target_Node})
// Create the relationship with dynamic type
CALL apoc.create.relationship(source, row.Pharmacists_Label, {name: row.Relationship, step: row.Step, subprocess: row.Subprocess}, target)
YIELD rel
RETURN rel;
""")

# Execute the queries
with driver.session() as session:
    driver.verify_connectivity()
    print("Connection established.")
    session.run(delete_data)
    session.run(load_nodes)
    session.run(load_relationships)
    print(f"Data loaded successfully.")

Connection established.
Data loaded successfully.


## Group the data
- Group the data by classes
- Group the data by EPA
- Give classes EPA labels

In [15]:
# Group by class
label_class = Query("""
MATCH (n)
WITH DISTINCT n.class AS class, collect(DISTINCT n) AS nodes
CALL apoc.create.addLabels(nodes, [apoc.text.upperCamelCase(class)]) YIELD node
RETURN *
""")

# Group by EPA
label_epa = Query("""
MATCH (n)
WITH DISTINCT n.epa_label AS label, collect(DISTINCT n) AS nodes
CALL apoc.create.addLabels(nodes, [apoc.text.upperCamelCase(label)]) YIELD node
RETURN *
""")

# Give EPA label
give_epa_label = Query("""
MATCH (n)
WHERE NOT(n.epa_label="nan")
SET n:EPA
""")

# Return Labels
return_labels = Query("""
MATCH (n)
RETURN DISTINCT labels(n) AS labels, count(*) AS count
""")

# Execute the queries
with driver.session() as session:
    session.run(label_class)
    session.run(label_epa)
    session.run(give_epa_label)
    result = session.run(return_labels)
    df = result.to_df()
    print(f"Labels:")
    display(df)

Labels:


,labels,count
0,"[Node, Medicationstatement, Nan]",27
1,"[Node, Time, Nan]",4
2,"[Node, Condition, Anamnese, EPA]",7
3,"[Node, Other, Nan]",11
4,"[Node, Diagnosis, Nan]",21
5,"[Node, Attribute, Nan]",23
6,"[Node, Diagnosis, AdmissionDiagnosis, EPA]",2
7,"[Node, Observation, Nan]",26
8,"[Node, Library, Nan]",8
9,"[Node, Questionnaireresponse, Nan]",4


## Display and explore Graph with yFiles

In [16]:
from google.colab import output
output.enable_custom_widget_manager()

from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
display_subprocess = Query("""
MATCH (n)-[r]->(m)
WHERE r.subprocess= "10"
RETURN n, r, m
""")

g = Neo4jGraphWidget(driver)
g.show_cypher(display_subprocess)

GraphWidget(layout=Layout(height='710px', width='100%'))

## Evaluation:



### Total Relationships per Node
set as new Property!

In [68]:
total_rels_per_node = Query("""
MATCH (n)
OPTIONAL MATCH (n)-[r]-()
WITH n, COUNT(r) AS totalRelationships
SET n.relsCount = totalRelationships
RETURN n.id, n.epa_label, totalRelationships
ORDER BY n.epa_label, totalRelationships DESCENDING
""")

with driver.session() as session:
    result = session.run(total_rels_per_node)
    #df1 = result.to_df()
    #display(df1)

# Fügt Property zu jedem Knoten hinzu: Pharm_X_Count, usw. die zählt wie häufig hier die Relationships ein und ausgehen
for count in range (1,6):
  Pharm_rels_per_node = Query(f"""
  MATCH (a)
  OPTIONAL MATCH (a)-[r:Pharmacist_{count}]-()
  WITH a, COUNT(r) AS Pharm_{count}_Rels
  SET a.pharm_{count}_rels = Pharm_{count}_Rels
  RETURN a.id, a.epa_label, Pharm_{count}_Rels
  ORDER BY a.epa_label, Pharm_{count}_Rels DESC
  """)

  # Execute the queries
  with driver.session() as session:
      result = session.run(Pharm_rels_per_node)

display_rels = Query(f"""
  MATCH (a)
  RETURN a.id, a.epa_label AS label, a.relsCount AS totalRels, a.pharm_1_rels, a.pharm_2_rels, a.pharm_3_rels, a.pharm_4_rels, a.pharm_5_rels
  ORDER BY label, totalRels DESC
  """)

with driver.session() as session:
    result = session.run(display_rels)
    df2 = result.to_df()
    display(df2)

,a.id,label,totalRels,a.pharm_1_rels,a.pharm_2_rels,a.pharm_3_rels,a.pharm_4_rels,a.pharm_5_rels
0,A0,Anamnese,36,2,2,18,4,10
1,A1,Anamnese,12,4,2,4,0,2
2,A2,Anamnese,11,2,3,4,0,2
3,A3,Anamnese,4,0,0,0,0,4
4,A6,Anamnese,2,0,0,0,2,0
...,...,...,...,...,...,...,...,...
204,VD0,prelim_diagnosis,23,4,2,7,2,8
205,VD2.2,prelim_diagnosis,9,2,0,0,0,7
206,VD2.1,prelim_diagnosis,2,0,0,0,0,2
207,S0,sonography,4,2,2,0,0,0


###  How many times per Pharmacist: EPA labels - Laboratory Values, Medication, etc.


In [59]:

display_rels_per_epa_class = Query(f"""
  MATCH (a)
  WITH a.epa_label AS label, SUM(a.totalRelationshipCount) as totalRels, SUM(a.Pharm_1_Rels) AS totalRels_1, SUM(a.Pharm_2_Rels) AS totalRels_2,SUM(a.Pharm_3_Rels) AS totalRels_3,SUM(a.Pharm_4_Rels) AS totalRels_4,SUM(a.Pharm_5_Rels) AS totalRels_5
  RETURN label, totalRels, totalRels_1, totalRels_2, totalRels_3, totalRels_4, totalRels_5
  ORDER BY label, totalRels DESC
  """)

with driver.session() as session:
    result = session.run(display_rels_per_epa_class)
    df1 = result.to_df()
    display(df1)


,label,totalRels,totalRels_1,totalRels_2,totalRels_3,totalRels_4,totalRels_5
0,Anamnese,69,8,7,30,6,18
1,admission_diagnosis,41,8,2,8,6,17
2,cell_count_in_urine,35,15,2,4,7,7
3,ekg,18,8,2,0,0,8
4,examination_findings,21,4,4,6,2,5
5,follow-up_findings,27,0,4,7,10,6
6,laboratory_values,189,19,20,37,50,63
7,medication,362,59,93,67,61,82
8,nan,1422,205,280,286,247,404
9,patient_information,35,2,0,4,13,16


### How many times per Pharmacist: Issues like needsRequest, needsClarification, needs